In [24]:
from components.modules.dual_dcf_model_main import portfolio_dist 

from components.modules.stock_forecast_main import stock_options_weights

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
import time
import re
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np

In [27]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [28]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

In [4]:
import requests
import json
from bs4 import BeautifulSoup
import os

In [5]:
API_TOKEN = "rxfMXxfsdsKGb8urL8shecBbBp37ZBuvU5zo39sm" # should change this to secret token
API_ENDPOINT = f"https://api.marketaux.com/v1/news/all"

In [6]:
payload = {
    "api_token": API_TOKEN,
    "entity_types": "equity,index,etf", # should include industries parameter
#     "industries": "Technology",
    "countries": "us",
    "published_after": "2024-05-01T13:00",
    "published_before": "2024-05-01T14:00",
    "domains": "finance.yahoo.com"
}

In [7]:
def get_urls(payload):
    """
    Retrieves URLs of articles from Yahoo Finance
    payload: dict, criteria for article selection
    return: list, Yahoo Finance article URLs
    """
    r = requests.get("https://api.marketaux.com/v1/news/all", params=payload)
    output = r.text
    json_output = json.loads(output)
    data = json_output['data']
    
    urls = []
    
    for i in range(len(data)):
        url = data[i]['url']
        if url.startswith("https://finance.yahoo.com/news"):
            urls.append(url)
        
    return urls

In [8]:
def get_article_content(url):
    """
    Retrieves article content
    url: str, article URL
    return: str, article content
    """
    r = requests.get(url)
    
    if r.status_code != 200:
        return 'Invalid Response'
    
    soup = BeautifulSoup(r.content)
    body = soup.find('div', attrs={'class': 'caas-body'})
    content = ""
    
    for text in body.find_all('p'):
        content += text.text
    
    return content

In [9]:
def write_to_file(filename, content):
    """
    Creates and writes article content to file
    filename: str, name of file to create
    content: str, content to write to file
    """
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(content)

In [10]:
def create_documents(start_date, num_days):
    """
    Retrieves article content over a number of days and saves the content to files in the 'articles' folder
    start_date: str, date in the format YYYY-MM-DD. articles published on this day are retrieved
    num_days: int, the number of days after start_date to retrieve articles for. i.e. if num_days=10, articles published 
        on the start_date and every day between the start_date and 10 days after are retrieved
    """
    # start_date looks like "2024-05-01"
    payload = {
        "api_token": API_TOKEN,
        "entity_types": "equity,index,etf", 
        "countries": "us",
#         "published_after": "2024-05-10T13:00", # what time format is supposed to look like, in UTC format 
#         "published_before": "2024-05-10T14:00",
        "domains": "finance.yahoo.com"
    }
    
    def get_dates(start_date, num_days):
        # Convert the input string to a datetime object
        date_obj = datetime.strptime(start_date, "%Y-%m-%d")
        dates = [date_obj.strftime('%Y-%m-%d')]
        # Print the input date
        print(date_obj.strftime("%Y-%m-%d"))

        # Loop to get a total of num_days days
        for i in range(1, num_days):
            next_day = date_obj + timedelta(days=i)
            dates.append(next_day.strftime('%Y-%m-%d'))
            print(f"{next_day.strftime('%Y-%m-%d')}")
        
        return dates
        
    dates = get_dates(start_date, num_days)
    
    for date in dates: 
        for add_hour in range(7): 
            start = date + "T" + str(13 + add_hour) + ":00" # edit str(xx) for start day
            finish = date + "T" + str(13 + add_hour + 1) + ":00"
            payload["published_after"] = start
            payload["published_before"] = finish
            
            urls = get_urls(payload)
            for i in range(len(urls)):
                print(urls[i])
                content = get_article_content(urls[i])
                if content != "Invalid Response": 
                    filename = os.path.join("..", "..", "new-articles", "article_" + start + "_" + str(add_hour) + str(i) + ".txt")
                    write_to_file(filename, content)
                else:
                    print("Above URL Skipped")

In [8]:
def get_llm(selected_model):
    """
    Initializes and stores LLM on memory
    selected_model: str, model name to load
    return: model variable
    """
    
    SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
    - Generate human readable output, avoid creating output with gibberish text.
    - Generate only the requested output, don't include any other language before or after the requested output.
    - Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
    - Generate professional language typically used in business documents in North America.
    - Never generate offensive or foul language.
    """

    query_wrapper_prompt = PromptTemplate(
        "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
    )

    llm = HuggingFaceLLM(
        context_window=4096,
        max_new_tokens=2048,
        generate_kwargs={"temperature": 0.1, "do_sample": True},
        query_wrapper_prompt=query_wrapper_prompt,
        tokenizer_name=selected_model,
        model_name=selected_model,
        device_map="auto",
        # change these settings below depending on your GPU
        model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True},
    )
        
    return llm

In [9]:
def get_query_engine(llm):
    """
    Creates the necessary components to run a RAG pipeline
    llm: LLM object loaded into memory
    return: SimpleDirectoryReader, VectorStoreIndex, QueryEngine. the first object houses the data, the second object indexes
        the data for fast retrieval, and the third object is how we can query the data using the LLM
    """
    set_global_tokenizer(
        AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
    )
    embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
    
    yf_documents = SimpleDirectoryReader(input_dir="new-articles").load_data()
    yf_index = VectorStoreIndex.from_documents(yf_documents, embed_model=embed_model)
    yf_query_engine = yf_index.as_query_engine(llm=llm, streaming=True)
    
    return yf_documents, yf_index, yf_query_engine

In [1]:
def get_stocks(query_engine, user_prefs):
    """
    Returns set of stocks to invest in based on customer preferences and market news
    llm: LLM object loaded into memory
    user_prefs: dict, has keys "num_stocks", "risk", "industries", "return_goals", contains information about user preferences
    return: list, set of stocks matching user preferences returned from LLM based on articles 
    """
    
    def get_prompt(user_prefs):
        """
        Creates a prompt specific to user preferences
        user_prefs: dict, has keys "num_stocks", "risk", "industries", "return_goals", contains information about user preferences
        return: str, prompt to query the llm with 
        """
        start_prompt = ("Gather stocks and ETFs that are mentioned across the articles the most in a positive sentiment."
             f" Return a list of {user_prefs['num_stocks']} these stocks or ETFs. Collect them so that no more than half "
                "are in the same industry for diversification purposes. Make sure you don't repeat any stocks. ")
        risk_averse = ("This list should be tailored to a very risk averse investor who "
            "is more focused on stability instead of rapid growth. This list should include large market cap stocks "
            "with good fundamentals and a history of good performance. ")
        risk_neutral = ""
        risk_tolerant = ("This list should also be tailored to a very risk tolerant investor who "
            "is more focused on rapid growth instead of stability. This list should include growth stocks with high expected returns " 
            "and the potential for greater risk. ")
        industries_prompt = f"Include multiple stocks with a strong focus on the following industry or industries: {user_prefs['industries']}. "
        end_prompt = ("Create this list so that the stocks can be put together to make a diversified portfolio. " 
                "List the stock name and stock ticker in parentheses. Only include in your output the stock name and ticker. " 
                "Do not include any other recommendations. Do not include any unnecessary output or rationale.")

        prompt_dict = {
            "risk_averse": risk_averse, 
            "risk_neutral": risk_neutral,
            "risk_tolerant": risk_tolerant
        }
        prompt = None

        if user_prefs["risk"] == "conservative":
            prompt = start_prompt + prompt_dict["risk_averse"]
        elif user_prefs["risk"] == "aggressive":
            prompt = start_prompt + prompt_dict["risk_tolerant"]
        else:
            prompt = start_prompt + prompt_dict["risk_neutral"]

        if user_prefs['industries']:
            prompt += industries_prompt

        prompt += end_prompt

        return prompt
    
    prompt = get_prompt(user_prefs)
    
    VALID = False
    attempts = 0
    
    while not VALID:
        attempts += 1
        print(attempts)
        response = query_engine.query(prompt)
        response.print_response_stream()
    
        pattern = r'\(([A-Za-z]+)\)'
        stocks = re.findall(pattern, response.response)
    
    
        for ticker in stocks:
            stock = yf.Ticker(ticker)
            if 'regularMarketOpen' not in stock.info:
                VALID = False
                break
            else:
                VALID = True
                
    return stocks

In [12]:
risk_options = ["conservative", "moderate", "aggressive"]
industry_options = ["Energy", "Materials", "Industrials", "Utilities", "Healthcare", "Financials", "Consumer Discretionary",
                    "Consumer Staples", "Technology", "Communication Services", "Real Estate"]
return_goals = []
user_prefs = {
    "num_stocks": None, 
    "risk": None, 
    "industries": None, 
}

In [13]:
LLAMA2_7B_CHAT = "meta-llama/Llama-2-7b-chat-hf"
selected_model = LLAMA2_7B_CHAT

In [14]:
# create_documents("2024-05-20", 5)

In [15]:
llm = get_llm(selected_model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
r = llm.complete("Write a Python function that only prints even numbers")

2024-05-28 19:21:27.621037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
print(r)

Of course! Here is a Python function that only prints even numbers:

def print_even_numbers():
    for number in range(1, 101):
        if number % 2 == 0:
            print(number)

I hope this helps! Let me know if you have any other questions.


In [18]:
docs, index, query_engine = get_query_engine(llm)

In [19]:
# example user preferences
user_prefs = {"num_stocks": 5, "risk": "conservative", "industries": None}

In [29]:
prompt = get_prompt(user_prefs)
print(prompt)
print()
r = query_engine.query(prompt)
r.print_response_stream()

Gather stocks and ETFs that are mentioned across the articles the most in a positive sentiment. Return a list of 5 these stocks or ETFs. Collect them so that no more than half are in the same industry for diversification purposes. Make sure you don't repeat any stocks. Create this list so that the stocks can be put together to make a diversified portfolio. List the stock name and stock ticker in parentheses. Only include in your output the stock name and ticker. Do not include any other recommendations. Do not include any unnecessary output or rationale.

Sure, I can help you with that! Based on the articles provided, the following are 5 stocks or ETFs that are mentioned in a positive sentiment:

1. Nvidia (NVDA)
2. Microsoft (MSFT)
3. ServiceNow (NOW)
4. Netflix (NFLX)
5. Booking Holdings (BKNG)

These stocks are from different industries, which meets the diversification requirement.

In [39]:
user_prefs = {"num_stocks": 5, "risk": "conservative", "industries": ["Energy"]}
prompt = get_prompt(user_prefs)
print(prompt)
print()
r = query_engine.query(prompt)
r.print_response_stream()

Gather stocks and ETFs that are mentioned across the articles the most in a positive sentiment. Return a list of 5 these stocks or ETFs. Collect them so that no more than half are in the same industry for diversification purposes. Make sure you don't repeat any stocks. This list should be tailored to a very risk averse investor who is more focused on stability instead of rapid growth. This list should include large market cap stocks with good fundamentals and a history of good performance. Include multiple stocks with a strong focus on the following industry or industries: ['Energy']. Create this list so that the stocks can be put together to make a diversified portfolio. List the stock name and stock ticker in parentheses. Only include in your output the stock name and ticker. Do not include any other recommendations. Do not include any unnecessary output or rationale.

Sure, I can help you with that! Based on the articles provided, the following are 5 stocks or ETFs that are mentione

In [40]:
user_prefs = {"num_stocks": 5, "risk": "conservative", "industries": ["Energy", "Healthcare"]}
prompt = get_prompt(user_prefs)
print(prompt)
print()
r = query_engine.query(prompt)
r.print_response_stream()

Gather stocks and ETFs that are mentioned across the articles the most in a positive sentiment. Return a list of 5 these stocks or ETFs. Collect them so that no more than half are in the same industry for diversification purposes. Make sure you don't repeat any stocks. This list should be tailored to a very risk averse investor who is more focused on stability instead of rapid growth. This list should include large market cap stocks with good fundamentals and a history of good performance. Include multiple stocks with a strong focus on the following industry or industries: ['Energy', 'Healthcare']. Create this list so that the stocks can be put together to make a diversified portfolio. List the stock name and stock ticker in parentheses. Only include in your output the stock name and ticker. Do not include any other recommendations. Do not include any unnecessary output or rationale.

Sure, I can help you with that! Based on the articles provided, the following are 5 stocks or ETFs tha

In [31]:
user_prefs = {"num_stocks": 5, "risk": "moderate", "industries": None}
prompt = get_prompt(user_prefs)
print(prompt)
print()
r = query_engine.query(prompt)
r.print_response_stream()

Gather stocks and ETFs that are mentioned across the articles the most in a positive sentiment. Return a list of 5 these stocks or ETFs. Collect them so that no more than half are in the same industry for diversification purposes. Make sure you don't repeat any stocks. Create this list so that the stocks can be put together to make a diversified portfolio. List the stock name and stock ticker in parentheses. Only include in your output the stock name and ticker. Do not include any other recommendations. Do not include any unnecessary output or rationale.

Sure, I can help you with that! Based on the articles provided, here are 5 stocks or ETFs that are mentioned in a positive sentiment:

1. Nvidia (NVDA)
2. Microsoft (MSFT)
3. ServiceNow (NOW)
4. Netflix (NFLX)
5. Booking Holdings (BKNG)

These stocks are from different industries, which should provide a good level of diversification in a portfolio.

In [30]:
user_prefs = {"num_stocks": 5, "risk": "aggressive", "industries": None}
prompt = get_prompt(user_prefs)
print(prompt)
print()
r = query_engine.query(prompt)
r.print_response_stream()

Gather stocks and ETFs that are mentioned across the articles the most in a positive sentiment. Return a list of 5 these stocks or ETFs. Collect them so that no more than half are in the same industry for diversification purposes. Make sure you don't repeat any stocks. This list should also be tailored to a very risk tolerant investor who is more focused on rapid growth instead of stability. This list should include growth stocks with high expected returns and the potential for greater risk. Create this list so that the stocks can be put together to make a diversified portfolio. List the stock name and stock ticker in parentheses. Only include in your output the stock name and ticker. Do not include any other recommendations. Do not include any unnecessary output or rationale.

Based on the provided articles, the following are the 5 stocks or ETFs that are mentioned the most in a positive sentiment:

1. Nvidia (NVDA)
2. Bloom Energy (BE)
3. Intel (INTC)
4. Advanced Micro Devices (AMD)


In [42]:
user_prefs = {"num_stocks": 5, "risk": "moderate", "industries": None}
prompt = get_prompt(user_prefs)
print(prompt)
print()
r = query_engine.query(prompt)
r.print_response_stream()

Gather stocks and ETFs that are mentioned across the articles the most in a positive sentiment. Return a list of 5 these stocks or ETFs. Collect them so that no more than half are in the same industry for diversification purposes. Make sure you don't repeat any stocks. Create this list so that the stocks can be put together to make a diversified portfolio. List the stock name and stock ticker in parentheses. Only include in your output the stock name and ticker. Do not include any other recommendations. Do not include any unnecessary output or rationale.

Sure, I can help you with that! Based on the articles provided, the following are 5 stocks that are mentioned in a positive sentiment:

1. Nvidia (NVDA)
2. Microsoft (MSFT)
3. ServiceNow (NOW)
4. Netflix (NFLX)
5. Booking Holdings (BKNG)

These stocks are diverse in terms of industry, with Nvidia and Microsoft being from the technology sector, ServiceNow and Netflix being from the software and media industries, and Booking Holdings be

In [43]:
stocks = ['NVDA', 'MSFT', 'NOW', 'NFLX', 'BKNG']

In [44]:
def pipeline_demo(stocks, ml_weight=0.5, fin_weight=0.5):
    """
    Modified pipeline that uses time series analysis and financial analysis to forecast performance and generate portfolio 
    weights
    stocks: list, stocks to be included in portfolio
    return: list, stock recommendations and corresponding portfolio weights
    """
    
    start = '2020-01-01'
    end = datetime.today().strftime('%Y-%m-%d')
    days_to_predict = 92
    stock_prediction = stock_options_weights(stocks, start, end, days_to_predict)
    
    stock_recs, stock_weights = portfolio_dist(stocks, user_prefs["risk"])
    
    print(f"Weights from forecasting: {stock_prediction}")
    print(f"Weights from financial analysis: {stock_weights}")

    portfolio_weights_fin = np.array(list(stock_weights.values()))
    portfolio_weights_ml = np.array(list(stock_prediction.values()))[:, 1]

    final_weights = portfolio_weights_ml * ml_weight + portfolio_weights_fin * fin_weight
    return list(zip(stock_weights.keys(), final_weights))

In [45]:
pipeline_demo(stocks)

21:17:53 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
21:17:53 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/3 [00:00<?, ?it/s]

21:17:53 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
21:17:53 - cmdstanpy - INFO - Chain [1] done processing
21:17:54 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
21:17:54 - cmdstanpy - INFO - Chain [1] done processing
21:17:54 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after pa

  0%|          | 0/3 [00:00<?, ?it/s]

21:17:55 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
21:17:56 - cmdstanpy - INFO - Chain [1] done processing
21:17:56 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
21:17:56 - cmdstanpy - INFO - Chain [1] done processing
21:17:57 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after pa

...

21:21:05 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
21:21:05 - cmdstanpy - INFO - Chain [1] done processing
21:21:06 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
21:21:06 - cmdstanpy - INFO - Chain [1] done processing
21:21:06 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after pa

Best parameters: {'changepoint_prior_scale': 0.01, 'changepoint_range': 0.2}, Best RMSE: 638.5556054249337


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
21:21:08 - cmdstanpy - INFO - Chain [1] done processing


NVDA Buy 5
MSFT Hold 0
NOW Buy 5
NFLX Unavailable Cash Flow: AccountsReceivableNetCurrent
NFLX Hold 0
BKNG Hold 0
Weights from forecasting: {'NVDA': (551.2783711438647, 0.13939753383339828), 'MSFT': (345.22755954900185, 0.48853884466609176), 'NOW': (641.983957600032, 0.18014661431915752), 'NFLX': (509.2347565637769, 0.1392794872878239), 'BKNG': (2997.4960948119424, 0.05263751989352844)}
Weights from financial analysis: {'NVDA': 0.5, 'MSFT': 0.0, 'NOW': 0.5, 'NFLX': 0.0, 'BKNG': 0.0}


[('NVDA', 0.31969876691669913),
 ('MSFT', 0.24426942233304588),
 ('NOW', 0.34007330715957873),
 ('NFLX', 0.06963974364391196),
 ('BKNG', 0.02631875994676422)]

In [41]:
stocks = ['NVDA', 'BE', 'INTC', 'AMD', 'TSLA'] 
pipeline_demo(stocks)

20:45:25 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
20:45:25 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/3 [00:00<?, ?it/s]

20:45:25 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
20:45:25 - cmdstanpy - INFO - Chain [1] done processing
20:45:26 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
20:45:26 - cmdstanpy - INFO - Chain [1] done processing
20:45:26 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after pa

  0%|          | 0/3 [00:00<?, ?it/s]

20:45:28 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
20:45:28 - cmdstanpy - INFO - Chain [1] done processing
20:45:29 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
20:45:29 - cmdstanpy - INFO - Chain [1] done processing
20:45:29 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after pa

...

20:48:53 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
20:48:53 - cmdstanpy - INFO - Chain [1] done processing
20:48:54 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
20:48:54 - cmdstanpy - INFO - Chain [1] done processing
20:48:55 - cmdstanpy - INFO - Chain [1] start processing
huggingface/tokenizers: The current process just got forked, after pa

Best parameters: {'changepoint_prior_scale': 0.025, 'changepoint_range': 0.8}, Best RMSE: 126.62718754948119


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
20:48:56 - cmdstanpy - INFO - Chain [1] done processing


NVDA Buy 4
BE Buy 2
INTC Buy 6
AMD Hold 0
TSLA Hold 0
Weights from forecasting: {'NVDA': (551.2783711438647, 0.016283324881469323), 'BE': (14.315351659410943, 0.5418732943612942), 'INTC': (40.510990856931, 0.30158840723131175), 'AMD': (135.7593226842115, 0.08361977787382965), 'TSLA': (193.77394789518223, 0.0566351956520951)}
Weights from financial analysis: {'NVDA': 0.3333333333333333, 'BE': 0.16666666666666666, 'INTC': 0.5, 'AMD': 0.0, 'TSLA': 0.0}


[('NVDA', 0.17480832910740132),
 ('BE', 0.3542699805139804),
 ('INTC', 0.4007942036156559),
 ('AMD', 0.04180988893691483),
 ('TSLA', 0.02831759782604755)]